# Mike Babb
# babb.mike@outlook.com
# Find anagrams
## Get the parent / child words of the top 5 word groups by character length

In [ ]:
# standard libraries
from collections import Counter
import os
import sqlite3

In [ ]:
# external
import json
import pandas as pd

In [ ]:
# custom
import _run_constants as rc
from part_00_file_db_utils import query_db

# WORD COUNTS BY WORD GROUP

In [ ]:
# join in word_group_id
sql = 'select * from word_counts;'
word_df = query_db(sql = sql, db_path=rc.db_path, db_name=rc.db_name)

In [ ]:
word_df.head()

In [ ]:
# turn this into the word_group_df
wg_df = word_df.drop_duplicates(subset=['word_group_id']).copy()


In [ ]:
wg_df.head()

In [ ]:
wg_df.shape

In [ ]:
# rank
wg_df['n_from_rank'] = wg_df[['n_chars', 'n_from_words']].groupby(['n_chars']).rank(method = 'first', ascending=False)
wg_df['n_to_rank'] = wg_df[['n_chars', 'n_to_words']].groupby(['n_chars']).rank(method = 'first',ascending=False)

# melt to get the word count
wc_df = pd.melt(frame = wg_df, id_vars = ['word_id', 'word_group_id', 'lcase', 'n_chars'],
                  value_vars = ['n_from_words','n_to_words'],
                  var_name = 'direction', value_name = 'n_words')
				  
				  
recode_dict = {'n_from_words':'from',
               'n_to_words':'to'}

wc_df['direction'] = wc_df['direction'].map(recode_dict)			   
				  
# melt to get the ranks
rank_df = pd.melt(frame = wg_df, id_vars = ['word_id', 'word_group_id', 'lcase', 'n_chars'],
                  value_vars = ['n_from_rank','n_to_rank'],
                  var_name = 'direction', value_name = 'word_rank')			  
				  
recode_dict = {'n_from_rank':'from',
               'n_to_rank':'to'}

rank_df['direction'] = rank_df['direction'].map(recode_dict)		   
				  				  
# overwrite the wg_df by joining
wg_df = pd.merge(left = wc_df, right = rank_df)				  

In [ ]:
wg_df.head()

In [ ]:
# select the top 5
curr_wg_df = wg_df.loc[wg_df['word_rank'] <= 5, :].copy()

In [ ]:
curr_wg_df.shape

In [ ]:
word_group_id_counter = Counter(word_df['word_group_id'])

In [ ]:
curr_wg_df['word_group_size'] = curr_wg_df['word_group_id'].map(word_group_id_counter)

In [ ]:
curr_wg_df.head()

In [ ]:
curr_wg_df = curr_wg_df.sort_values(by = ['n_chars', 'lcase'])
temp_output_df = curr_wg_df[['lcase', 'n_chars', 'direction', 'n_words', 'word_rank', 'word_group_size']]

temp_output_df.columns = ['word', 'number of characters','direction', 'number of words', 'word rank', 'word group size']
temp_output_df.to_json('../webpage/word_groups.json',orient='records')

In [ ]:
curr_wg_df.shape

In [ ]:
curr_wg_df.head()

In [ ]:
# the from word id df
sql = 'select word_group_id, word_id, lcase, n_chars from words;'
fw_df = query_db(sql = sql, db_path=rc.db_path, db_name=rc.db_name)
fw_df.columns = ['from_word_group_id', 'from_word_id', 'from_word', 'from_n_chars']

# the to word id df
sql = 'select word_group_id, word_id, lcase, n_chars from words;'
tw_df = query_db(sql = sql, db_path=rc.db_path, db_name=rc.db_name)
tw_df.columns = ['to_word_group_id', 'to_word_id', 'to_word', 'to_n_chars']

In [ ]:
tw_df.head()

In [ ]:
# output path
word_list_output_path = '../webpage/wordlists'

In [ ]:
def save_words(df:pd.DataFrame, r_direction:str, curr_word:str):
    if r_direction == 'from':
        r_direction_opp = 'to'
    else:
        r_direction_opp = 'from'
    # the output file and name
    output_file_name = f'{r_direction}_{curr_word}.txt'
    ofpn = os.path.join(word_list_output_path, output_file_name)
    
    with open(file = ofpn, mode = 'w') as my_file:
        if df.empty:            
            my_file.write(f"## THERE ARE NO {r_direction} WORDS FOR {curr_word} ## \n")
        else:

            # number of characters to iterate through
            nchar_list = sorted(df[f'{r_direction_opp}_n_chars'].unique().tolist(), reverse=True)
        
            for nc in nchar_list:
                my_file.write(f"##########\n")
                my_file.write(f"## WORDS OF LENGTH {nc} ## \n")
                my_file.write(f"##########\n")
                # subset by 
                temp_df = df.loc[df[f'{r_direction_opp}_n_chars' ]==nc, [f'{r_direction_opp}_word']].copy()
                temp_df = temp_df.drop_duplicates().sort_values(by = f'{r_direction_opp}_word')
                for fw in temp_df[f'{r_direction_opp}_word'].tolist():
                    my_file.write(fw + '  \n')

In [ ]:
def save_words_json(df:pd.DataFrame, r_direction:str, curr_word:str):
    
    
    # the output file and name
    output_file_name = f'{curr_word}.json'
    ofpn = os.path.join(word_list_output_path, r_direction, output_file_name)
    
    with open(file = ofpn, mode = 'w') as my_file:
        
        if df.empty:            
            output_dict = {'word':curr_word,
                       'number_of_words':0,
                       'relatedWords':[]}        
            json.dump(obj = output_dict, fp =  my_file, indent=4)            
        else:
            df[f'{r_direction}_n_chars'] = df[f'{r_direction}_word'].str.len()
            col_names = [f'{r_direction}_n_chars', f'{r_direction}_word']            
            temp_df = df.sort_values(by=col_names, ascending=[False, True])
            word_list = temp_df[f'{r_direction}_word'].tolist()
            #if curr_word in word_list:
            #    word_list.remove(curr_word)

            output_dict = {'word':curr_word,
                           'number_of_words':temp_df.shape[0],
                           'relatedWords':word_list}        
            json.dump(obj = output_dict, fp =  my_file, indent=4)            
        
            

In [ ]:
col_names = ['word_group_id', 'lcase']
temp_word_df = word_df[col_names].sort_values(by = col_names).drop_duplicates(subset = 'word_group_id')

In [ ]:
# dictionary of words!
word_group_dict = {wg_id:word for wg_id, word in zip(temp_word_df['word_group_id'],
                                                     temp_word_df['lcase'])}

In [ ]:
def build_list_of_parent_words(word_group_id:int, db_path:str, db_name:str):

    # build the list of parent words
    sql = f'select from_word_group_id, to_word_group_id from anagram_groups where to_word_group_id = {word_group_id};'
    
    pwg_df = query_db(sql = sql, db_path=db_path, db_name=db_name)    

    # now, get the word list
    sql = 'select word_id as from_word_id, word_group_id as from_word_group_id, lcase as from_word from words;'
    word_df = query_db(sql = sql, db_path=db_path, db_name=db_name)
    pw_df = pd.merge(left = word_df, right = pwg_df)       
    
    # let's add information to highlight the focal word
    col_names = ['from_word_group_id', 'from_word_id', 'from_word']
    
    id_df = pw_df.loc[pw_df['from_word_group_id'] == word_group_id, col_names].copy()

    id_df.columns = ['to_word_group_id', 'to_word_id', 'to_word']
    pw_df = pd.merge(left = pw_df, right = id_df)

    col_names = ['from_word_id', 'to_word_id',
                 'from_word_group_id','to_word_group_id',
                 'from_word', 'to_word']
    
    pw_df = pw_df[col_names].drop_duplicates(subset=['from_word_id', 'from_word_group_id', 'from_word'])
    
    return pw_df

def build_list_of_child_words(word_group_id:int, db_path:str, db_name:str):

    # build the list of parent words
    sql = f'select from_word_group_id, to_word_group_id from anagram_groups where from_word_group_id = {word_group_id};'
    
    cwg_df = query_db(sql = sql, db_path=db_path, db_name=db_name)

    # now, get the word list
    sql = 'select word_id as to_word_id, word_group_id as to_word_group_id, lcase as to_word from words;'
    word_df = query_db(sql = sql, db_path=db_path, db_name=db_name)

    cw_df = pd.merge(left = word_df, right = cwg_df)
        
    # let's add information to highlight the focal word
    col_names = ['to_word_group_id', 'to_word_id', 'to_word']

    id_df = cw_df.loc[cw_df['to_word_group_id'] == word_group_id, col_names].copy()
    
    id_df.columns = ['from_word_group_id', 'from_word_id', 'from_word']
    cw_df = pd.merge(left = cw_df, right = id_df)
    
    col_names = ['from_word_id', 'to_word_id',
                 'from_word_group_id','to_word_group_id',
                 'from_word', 'to_word']
    
    cw_df = cw_df[col_names].drop_duplicates(subset=['to_word_id', 'to_word_group_id', 'to_word'])
    

    return cw_df

In [ ]:
wg_id = 60945
pw_df = build_list_of_parent_words(word_group_id=wg_id, db_path=rc.db_path, db_name=rc.db_name)
cw_df = build_list_of_child_words(word_group_id=wg_id, db_path=rc.db_path, db_name=rc.db_name)

In [ ]:
curr_wg_df.head()

In [ ]:
# counters
to_word_counter = Counter()
from_word_counter = Counter()
#test_wg_df = curr_wg_df.iloc]
for i_c, row in curr_wg_df.iterrows():
    

    # the current word
    curr_word_group_id = row['word_group_id']
    curr_word = word_group_dict[curr_word_group_id]    
    print(curr_word)

    if row['direction'] == 'to':
        print('#### TO WORDS')        
        
        cw_df = build_list_of_child_words(word_group_id=curr_word_group_id,
                                          db_path = rc.db_path, db_name = rc.db_name)        
        
        # save it
        save_words_json(df = cw_df, r_direction='to', curr_word=curr_word)
        
        # distinct from words
        #from_word_counter.update(wg_df['from_word_id'])

    if row['direction'] == 'from':
        print('#### FROM WORDS')
        
        pw_df = build_list_of_parent_words(word_group_id=curr_word_group_id,
                                          db_path = rc.db_path, db_name = rc.db_name)        
        
                
        # save it
        save_words_json(df = pw_df, r_direction='from', curr_word=curr_word)
        

        # distinct to words
        #to_word_counter.update(wg_df['to_word_id'])



In [ ]:
wg_df.head()

In [ ]:
word_df.head()

In [ ]:
# the letters in the top five words by word length can be found in:
print(len(from_word_counter)) 
# which is:
print(len(from_word_counter) / word_df.shape[0])
# of words



In [ ]:
# the letters in the top five words by word length can be rearraged to spell:
print(len(to_word_counter))
# which is:
print(len(to_word_counter) / word_df.shape[0])
# of words


In [ ]:
# what letters are represented?
from_letter_counter = Counter()
to_letter_counter = Counter()


In [ ]:
for ir, row in curr_wg_df.iterrows():
    if row['direction'] == 'from':
        from_letter_counter.update(row['lcase'])
    if row['direction'] == 'to':
        to_letter_counter.update(row['lcase'])

In [ ]:
from string import ascii_lowercase

In [ ]:
from_letter_counter

In [ ]:
set(ascii_lowercase).difference(from_letter_counter.keys())

In [ ]:
curr_wg_df.shape

In [ ]:
set(ascii_lowercase).difference(to_letter_counter.keys())

In [ ]:
to_letter_counter